<div style='float: right'><img src='pic/mashu.png'/></div>
## <div id='mashu' />ましゅ

In [ ]:
import numpy as np
from itertools import product
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
*.o..oo...
...o....oo
*.*o..o...
...*.*o...
ooo..*.o*.
......*.*.
.*..o....o
.....*o...
.ooo.o*oo.
......o...""".split()
nw, nh = len(data[0]), len(data)
M = nw * nh - 1
r2 = range(2)

### 問題
* 盤面に線を引き、すべての白丸と黒丸を通る1つの輪を作ります
* 線はタテヨコにマスの中央を通り、1マスに1本だけ通過できます
* 線をワクの外に出したり、交差や枝分かれさせたりしてはいけません
* 白丸を通る線は、白丸のマスで必ず直進し、白丸の両隣のマスの少なくとも片方で直角に曲がります
* 黒丸を通る線は、黒丸のマスで必ず直角に曲がりますが、黒丸の隣のマスで曲がってはいけません

### 変数
* vz：始点からの距離 (1)
* vh：水平線を引くかどうか (2)
* vv：垂直線を引くかどうか (3)
* vhd：0:to left, 1:to right (4)
* vvd：0:to down, 1:to up (5)

### 制約
* vzはM(マス数)以下 (6)
* 点に入るのは1以下 (7)
* 点に入る数と出る数は等しくなること (8)
* ○の条件。●の条件 (9)
* vzの更新式。vh、vvをvhd、vvdで表現 (10)

In [ ]:
m = LpProblem()
vz = addvars(nh, nw) # (1)
vh = addvars(nh, nw-1) # (2)
vv = addvars(nh-1, nw) # (3)
vhd = addbinvars(nh, nw-1, 2) # 0:to left, 1:to right (4)
vvd = addbinvars(nh-1, nw, 2) # 0:to down, 1:to up (5)
def dirs(i, j, k):
    return ([vhd[i][j-l][k^l] for l in r2 if 0 <= j-l < nw-1] +
            [vvd[i-l][j][k^l] for l in r2 if 0 <= i-l < nh-1])
for i, j in product(range(nh), range(nw)):
    m += vz[i][j] <= M # (6)
    din = dirs(i, j, 1)
    dout = dirs(i, j, 0)
    m += lpSum(din) <= 1 # (7)
    m += lpSum(din) == lpSum(dout) # (8)
    if data[i][j] == 'o':
        fy, fx = i, j
        m += lpSum(dirs(i, j, 1)) == 1 # (9)
        if 1 <= i < nh-1:
            m += vv[i-1][j] == vv[i][j] # (9)
        if 2 <= i < nh-2:
            m += vv[i-2][j] + vv[i+1][j] <= 2 - vv[i][j] # (9)
        if 1 <= j < nw-1:
            m += vh[i][j-1] == vh[i][j] # (9)
        if 2 <= j < nw-2:
            m += vh[i][j-2] + vh[i][j+1] <= 2 - vh[i][j] # (9)
    elif data[i][j] == '*':
        m += lpSum(vh[i][j-l] for l in r2 if 0 <= j-l < nw-1) == 1 # (9)
        if 0 <= i-2:
            m += vv[i-1][j] <= vv[i-2][j] # (9)
        if i+1 < nh - 1:
            m += vv[i][j] <= vv[i+1][j] # (9)
        if 0 <= j-2:
            m += vh[i][j-1] <= vh[i][j-2] # (9)
        if j+1 < nw-1:
            m += vh[i][j] <= vh[i][j+1] # (9)
for i, j in product(range(nh), range(nw-1)):
    m += lpSum(vhd[i][j]) == vh[i][j] # (10)
    m += vh[i][j] <= 1 # (10)
    m += vz[i][j] + 1 <= vz[i][j+1] + M * (1 - vhd[i][j][0]) # (10)
    if (i, j) != (fy, fx):
        m += vz[i][j+1] + 1 <= vz[i][j] + M * (1 - vhd[i][j][1]) # (10)
for i, j in product(range(nh-1), range(nw)):
    m += lpSum(vvd[i][j]) == vv[i][j] # (10)
    m += vv[i][j] <= 1 # (10)
    m += vz[i][j] + 1 <= vz[i+1][j] + M * (1 - vvd[i][j][0]) # (10)
    if (i, j) != (fy, fx):
        m += vz[i+1][j] + 1 <= vz[i][j] + M * (1 - vvd[i][j][1]) # (10)
%time m.solve()
for i in range(nh):
    for j in range(nw):
        print('+' if data[i][j]=='.' else data[i][j], end='')
        if j < nw-1:
            print('-' if value(vh[i][j]) else ' ', end='')
    print()
    if i == nh-1: break
    for j in range(nw):
        print('| ' if value(vv[i][j]) else '  ', end='')
    print()